# Draft 1 

**Author**: Lina M. Ramírez 

**First Modification**: June 7, 20222

**Last modification**: August 2, 2022

Dataset with 3927 obs, downloaded on April 27, 2022









In [5]:
#Import packages, import dataset, set options. 
import pandas as pd 
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import ttest_ind
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns

%cd "/Users/bfiuser/Dropbox/Chicago/UChicago/dip/data"
df = pd.read_csv('modified.csv')
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 85)

/Users/bfiuser/Dropbox/Chicago/UChicago/dip/data


,Unnamed: 0,year_of_birth,nationality,municipality,id_last_four,last_name_initial,email,form,score,video,fake_news_a,fake_news_b,fake_news_c,fake_news_d,fake_news_e,fake_news_f,fake_news_g,fake_news_h,fake_news_i,fake_news_j,fake_news_k,fake_news_l,fake_news_m,fake_news_n,fake_news_o,fake_news_p,fake_news_q,fake_news_r,fake_news_s,tweet_a_lack_of_information,tweet_a_report,tweet_b_lack_of_information,tweet_b_report,headline_1,headline_2,headline_3,headline_4,headline_5,headline_6,headline_7,headline_8,trust_1,trust_2,trust_3,trust_4,trust_5,trust_6,trust_7,trust_8,trust_9,ie_discrimination_a,ie_discrimination_b,ie_discrimination_c,ie_discrimination_d,ie_discrimination_e,ie_discrimination_f,ie_ambiguity_tolerance_a,ie_ambiguity_tolerance_b,ie_ambiguity_tolerance_c,ie_ambiguity_tolerance_d,ie_ambiguity_tolerance_e,ie_ambiguity_tolerance_f,ie_ambiguity_tolerance_g,ie_ambiguity_tolerance_h,ie_ambiguity_tolerance_i,ie_ambiguity_tolerance_j,ie_dehumanization_left,ie_dehumanization_center,ie_dehumanization_right,duration,date,age,treatment_control,treatment_only_f_ambiguity,treatment_only_f_dehumanization,treatment_only_f_discrimination,treatment_only_f_trust,treatment_only_v_ambiguity,treatment_only_v_dehumanization,treatment_only_v_discrimination,treatment_only_v_trust,treatment_v+f_ambiguity,treatment_v+f_dehumanization,treatment_v+f_discrimination,treatment_v+f_trust,male,female,gender_Otro,high_school,postgraduate,primary_school,technical_educ,undergraduate,social_stratum_Estrato 1,social_stratum_Estrato 2,social_stratum_Estrato 3,social_stratum_Estrato 4,social_stratum_Estrato 5,social_stratum_Estrato 6,social_stratum_Rural,department_Amazonas,department_Antioquia,department_Arauca,"department_Archipiélago de San Andrés, Providencia y Santa Catalina",department_Atlántico,bog_dept,department_Bolívar,department_Boyacá,department_Caldas,department_Caquetá,department_Casanare,department_Cauca,department_Cesar,department_Chocó,department_Cundinamarca,department_Córdoba,department_Guainía,department_Guaviare,department_Huila,department_La Guajira,department_Magdalena,department_Meta,department_Nariño,department_Norte de Santander,department_Putumayo,department_Quindío,department_Risaralda,department_Santander,department_Sucre,department_Tolima,department_Valle del Cauca,any_treatment,any_form,any_video,any_video_form,fake_right,fake_left,fake_center,true_right,true_left,true_center,all_fake,all_true,d_fake_right_reliable,d_fake_left_reliable,d_fake_center_reliable,d_true_right_reliable,d_true_left_reliable,d_true_center_reliable,d_all_fake_reliable,d_all_true_reliable,d_fake_right_unreliable,d_fake_left_unreliable,d_fake_center_unreliable,d_true_right_unreliable,d_true_left_unreliable,d_true_center_unreliable,d_all_fake_unreliable,d_all_true_unreliable,ambiguity_order_necessity,ambiguity_aversion,ambiguity_decision,d_ambiguity_order_necessity,d_ambiguity_aversion,d_ambiguity_decision,discrimination_index,d_discrimination,d_negative_emotions_left,d_negative_emotions_right,d_negative_emotions_center,d_positive_emotions_left,d_positive_emotions_right,d_positive_emotions_center,d_trust_1,d_trust_2,d_trust_3,d_trust_6,d_trust_7,d_trust_8,d_trust_9
0,0,1985,Belice,Bogotá D.C.,0,B,beatrivallejo@gmail.com,NaN,NaN,No,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,1,1,1,1,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,5,5,5,1,1,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,"Educación,Agitación,Tensión,Inquietud,Arrepent...","Emoción,Timidez,Educación,Sorpresa,Impresión,V...","Imprudencia,Aburrición,Nostalgia,Orgullo,Impre...",NaN,NaN,37,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,5.00,5.00,5.0,5.000000,5.000000,5.000000,5.0,5.000000,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,5.0,5.0,5.000000,0,0,0,5.000000,0,0,1,1,1,1,1,1,1,1,1,1,1,1
1,1,1976,Colombia,Bogotá D.C.,8208,V,mavillateg@gmail.com,Tolerancia a la ambigüedad,3.00,Sí,2,2,2,2,5,4,1,1,4,1,2,1,1,1,1,2,3,3,3,1,1,1,1,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0

## 1. Fake News - Reliable

**Regression Model 2** 

$ Y = \alpha + \beta_1 D_1 + \beta_2 D_2 + \beta_3 D_1 \times D_2 + \gamma C + \mu $

$Y =$fake reliable; true reliable (D = 1 if 4 or 5, 0 otherwise)

$D_1 = 1$ if any video.

$D_2 = 1$ if any form. 

$C =$ Controls: Age, Gender, Education level and Department (Bog vs. the rest)

In [2]:
any_treatment_grp = df.groupby(['any_treatment'])
#Control Means 
any_treatment_grp[['d_fake_right_reliable', 'd_true_right_reliable', 'd_fake_center_reliable', 'd_true_center_reliable', 'd_fake_left_reliable', 'd_true_left_reliable', 'd_all_fake_reliable', 'd_all_true_reliable']].mean()

,d_fake_right_reliable,d_true_right_reliable,d_fake_center_reliable,d_true_center_reliable,d_fake_left_reliable,d_true_left_reliable,d_all_fake_reliable,d_all_true_reliable
any_treatment,,,,,,,,
0.0,0.197895,0.244211,0.245263,0.208421,0.221053,0.238947,0.163158,0.148421
1.0,0.175969,0.210908,0.204516,0.188326,0.175969,0.236046,0.149553,0.132510


In [5]:
#Declaring the variables of the model
D1 = df[['any_video']]
D2 = df[['any_form']]
D3 = df[['any_video_form']]
D = D1.join(D2)
D = D.join(D3)
C = df[['age', 'female', 'high_school', 'postgraduate', 'technical_educ', 'undergraduate', 'bog_dept']]
X = D.join(C)
X = sm.add_constant(X)


Y1 = df['d_fake_right_reliable']
Y2 = df['d_true_right_reliable']
Y3 = df['d_fake_center_reliable'] 
Y4 = df['d_true_center_reliable']
Y5 = df['d_fake_left_reliable']
Y6 = df['d_true_left_reliable']
Y7 = df['d_all_fake_reliable']
Y8 = df['d_all_true_reliable']


In [41]:
def coef(Yvar): 
    
    model = sm.OLS(Yvar, X)
    est = model.fit()
    
    coef = pd.DataFrame(est.summary().tables[1].data)

# Column name 
    coef.columns = coef.iloc[0]

# Drop the extra row with column labels
    coef_=coef.drop(0)


# Keep only the any_treatment 
    coef.drop(coef.index[[0, 1, 5, 6, 7, 8, 9, 10, 11]], inplace = True)


# Set index to variable names 
    coef = coef.set_index(coef.columns[0])

# Change datatype from object to float
    coef = coef.astype(float)

    coef.rename(index={'any_video':'Video', 'any_form':'Form', 'any_video_form':'Both'},inplace=True)

    return coef

**Fake News Right**

In [5]:
coef(Y1)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,-0.0322,0.020,-1.630,0.103,-0.071,0.007
Form,0.0053,0.017,0.316,0.752,-0.028,0.038
Both,-0.0458,0.019,-2.380,0.017,-0.084,-0.008


In [6]:
model = sm.OLS(Y1, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)
r2

0.047647808229416944

**True News Right**

In [7]:
coef(Y2) 

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,-0.0365,0.022,-1.695,0.090,-0.079,0.006
Form,-0.0151,0.018,-0.829,0.407,-0.051,0.021
Both,-0.0538,0.021,-2.573,0.010,-0.095,-0.013


In [8]:
model = sm.OLS(Y2, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2


0.024059206621531937

**Fake News Center**

In [9]:
coef(Y3) 

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,-0.0483,0.021,-2.295,0.022,-0.090,-0.007
Form,-0.0130,0.018,-0.729,0.466,-0.048,0.022
Both,-0.0670,0.020,-3.272,0.001,-0.107,-0.027


In [10]:
model = sm.OLS(Y3, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2


0.052288514896191574

**True News Center**

In [11]:
coef(Y4)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,-0.0184,0.020,-0.908,0.364,-0.058,0.021
Form,0.0045,0.017,0.265,0.791,-0.029,0.038
Both,-0.0469,0.020,-2.379,0.017,-0.086,-0.008


In [12]:
model = sm.OLS(Y4, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.04756044421875216

**Fake News Left**

In [13]:
coef(Y5)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,-0.0333,0.020,-1.655,0.098,-0.073,0.006
Form,-0.0231,0.017,-1.355,0.175,-0.056,0.010
Both,-0.0761,0.020,-3.890,0.000,-0.114,-0.038


In [14]:
model = sm.OLS(Y5, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.041843319065658635

**True News Left**

In [15]:
coef(Y6)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0274,0.022,1.237,0.216,-0.016,0.071
Form,0.0106,0.019,0.566,0.571,-0.026,0.047
Both,-0.0421,0.022,-1.955,0.051,-0.084,0.000


In [16]:
model = sm.OLS(Y6, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2


0.014381026978995903

**All fake**

In [17]:
coef(Y7)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,-0.0081,0.018,-0.436,0.663,-0.044,0.028
Form,0.0078,0.016,0.500,0.617,-0.023,0.038
Both,-0.0395,0.018,-2.200,0.028,-0.075,-0.004


In [18]:
model = sm.OLS(Y7, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2


0.046305610422964216

**All true**

In [19]:
coef(Y8) 

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,-0.0153,0.018,-0.868,0.386,-0.050,0.019
Form,0.0084,0.015,0.560,0.575,-0.021,0.038
Both,-0.0430,0.017,-2.507,0.012,-0.077,-0.009


In [20]:
model = sm.OLS(Y8, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.0477150399009133

## 2. Fake News - Unreliable 

In [4]:
any_treatment_grp = df.groupby(['any_treatment'])
#Control Means 
any_treatment_grp[['d_fake_right_unreliable', 'd_true_right_unreliable', 'd_fake_center_unreliable', 'd_true_center_unreliable', 'd_fake_left_unreliable', 'd_true_left_unreliable', 'd_all_fake_unreliable', 'd_all_true_unreliable']].mean()

,d_fake_right_unreliable,d_true_right_unreliable,d_fake_center_unreliable,d_true_center_unreliable,d_fake_left_unreliable,d_true_left_unreliable,d_all_fake_unreliable,d_all_true_unreliable
any_treatment,,,,,,,,
0.0,0.257895,0.164211,0.293684,0.226316,0.205263,0.177895,0.130526,0.189474
1.0,0.272262,0.187047,0.340435,0.272689,0.240733,0.170430,0.138475,0.225394


In [6]:
Y1 = df['d_fake_right_unreliable']
Y2 = df['d_true_right_unreliable']
Y3 = df['d_fake_center_unreliable'] 
Y4 = df['d_true_center_unreliable']
Y5 = df['d_fake_left_unreliable']
Y6 = df['d_true_left_unreliable']
Y7 = df['d_all_fake_unreliable']
Y8 = df['d_all_true_unreliable']


**Fake News Right**

In [8]:
coef(Y1)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0691,0.022,3.118,0.002,0.026,0.113
Form,-0.0275,0.019,-1.470,0.142,-0.064,0.009
Both,0.0132,0.022,0.614,0.539,-0.029,0.056


In [9]:
model = sm.OLS(Y1, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)
r2

0.09168177232876362

**True News Right**

In [10]:
coef(Y2)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0397,0.020,1.982,0.048,0.000,0.079
Form,0.0020,0.017,0.115,0.908,-0.031,0.035
Both,0.0364,0.019,1.868,0.062,-0.002,0.075


In [11]:
model = sm.OLS(Y2, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)
r2

0.013860754118361851

**Fake News Center**

In [13]:
coef(Y3)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0772,0.023,3.349,0.001,0.032,0.122
Form,0.0013,0.020,0.068,0.945,-0.037,0.040
Both,0.0615,0.022,2.743,0.006,0.018,0.105


In [14]:
model = sm.OLS(Y3, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)
r2

0.12324740426088221

**True News Center**

In [15]:
coef(Y4)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0740,0.022,3.351,0.001,0.031,0.117
Form,0.0117,0.019,0.626,0.531,-0.025,0.048
Both,0.0511,0.021,2.381,0.017,0.009,0.093


In [16]:
model = sm.OLS(Y4, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)
r2

0.07923845824304132

**Fake News Left**

In [17]:
coef(Y5)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0454,0.020,2.219,0.027,0.005,0.085
Form,0.0055,0.017,0.319,0.750,-0.028,0.039
Both,0.0423,0.020,2.126,0.034,0.003,0.081


In [18]:
model = sm.OLS(Y5, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)
r2

0.14470766584665362

**True News Left**

In [19]:
coef(Y6) 

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,-0.0061,0.020,-0.311,0.756,-0.044,0.032
Form,-0.0218,0.017,-1.317,0.188,-0.054,0.011
Both,-0.0005,0.019,-0.029,0.977,-0.038,0.037


In [21]:
model = sm.OLS(Y6, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)
r2

0.026386129423924598

**All Fake**

In [22]:
coef(Y7)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0265,0.018,1.490,0.136,-0.008,0.061
Form,-0.0178,0.015,-1.185,0.236,-0.047,0.012
Both,0.0238,0.017,1.377,0.169,-0.010,0.058


In [23]:
model = sm.OLS(Y7, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)
r2

0.02456828497605501

**All True**

In [24]:
coef(Y8) 

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0643,0.020,3.190,0.001,0.025,0.104
Form,-0.0003,0.017,-0.018,0.986,-0.034,0.033
Both,0.0429,0.020,2.189,0.029,0.004,0.081


In [25]:
model = sm.OLS(Y8, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)
r2

0.12750169413200096

## 3. Tweets 

In [21]:
Y1 = df['tweet_a_lack_of_information']
Y2 = df['tweet_b_lack_of_information']
Y3 = df['tweet_a_report']
Y4 = df['tweet_b_report']

#Declaring the variables of the model
D1 = df[['any_video']]
D2 = df[['any_form']]
D3 = df[['any_video_form']]
D = D1.join(D2)
D = D.join(D3)
X = D.join(C)
X = sm.add_constant(X)

In [22]:
any_treatment_grp[['tweet_a_lack_of_information', 'tweet_a_report', 'tweet_b_lack_of_information', 'tweet_b_report']].mean()

,tweet_a_lack_of_information,tweet_a_report,tweet_b_lack_of_information,tweet_b_report
any_treatment,,,,
0.0,0.730526,0.592632,0.70000,0.581053
1.0,0.747763,0.646783,0.71666,0.606306


In [23]:
# Tweet a - Question 1 
coef_1 = coef(Y1)

In [24]:
model = sm.OLS(Y1, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2


0.02179727761922956

In [25]:
# Tweet b - Question 1
coef_2 = coef(Y2)

In [26]:
model = sm.OLS(Y2, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2


0.024728087075201355

In [27]:
# Tweet a - Question 2
coef_3 = coef(Y3)

In [28]:
model = sm.OLS(Y3, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.005124124934685947

In [29]:
# Tweet b - Question 2
coef_4 = coef(Y4)

In [30]:
model = sm.OLS(Y4, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.004325618354710792

In [31]:
#Append coefficients and show estimates. 
coef_df = pd.concat([coef_1, coef_2, coef_3, coef_4])

In [32]:
# Show estimates 
coef_df.head(15)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0323,0.023,1.424,0.155,-0.012000,0.077
Form,0.0015,0.019,0.078,0.938,-0.036000,0.039
Both,0.0188,0.022,0.852,0.394,-0.024000,0.062
Video,0.0264,0.023,1.125,0.261,-0.020000,0.072
Form,0.0046,0.020,0.230,0.818,-0.034000,0.044
Both,0.0166,0.023,0.725,0.469,-0.028000,0.061
Video,0.0681,0.025,2.696,0.007,0.019000,0.118
Form,0.0419,0.021,1.960,0.050,-0.000008,0.084
Both,0.0623,0.025,2.536,0.011,0.014000,0.110


## 4. Trust 

In [33]:
Y1 = df['d_trust_1']
Y2 = df['d_trust_2']
Y3 = df['d_trust_3']
Y4 = df['trust_4']
Y5 = df['trust_5']
Y6 = df['d_trust_6']
Y7 = df['d_trust_7']
Y8 = df['d_trust_8']
Y9 = df['d_trust_9']

#Declaring the variables of the model
D1 = df[['any_video']]
D2 = df[['any_form']]
D3 = df[['any_video_form']]
D = D1.join(D2)
D = D.join(D3)
X = D.join(C)
X = sm.add_constant(X)

In [34]:
any_treatment_grp[['d_trust_1', 'd_trust_2', 'd_trust_3', 'trust_4', 'trust_5', 'd_trust_6', 'd_trust_7', 'd_trust_8', 'd_trust_9']].mean()

,d_trust_1,d_trust_2,d_trust_3,trust_4,trust_5,d_trust_6,d_trust_7,d_trust_8,d_trust_9
any_treatment,,,,,,,,,
0.0,0.756842,0.753684,0.747368,0.593684,0.746316,0.105263,0.111579,0.108421,0.136842
1.0,0.780997,0.757137,0.758415,0.629740,0.748615,0.096293,0.092885,0.097997,0.109075


In [35]:
coef_1 = coef(Y1)
# Question 1
model = sm.OLS(Y1, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.012173498769978197

In [36]:
coef_2 = coef(Y2)
# Question 2
model = sm.OLS(Y2, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.007335624871845692

In [37]:
coef_3 = coef(Y3)
# Question 3
model = sm.OLS(Y3, X)
est = model.fit()
#est.summary()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.00730729118765594

In [38]:
coef_4 = coef(Y4)
# Question 3
model = sm.OLS(Y4, X)
est = model.fit()
#est.summary()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.01373379293728616

In [39]:
coef_5 = coef(Y5)
# Question 3
model = sm.OLS(Y5, X)
est = model.fit()
#est.summary()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.007677261034060634

In [40]:
coef_6 = coef(Y6)
# Question 3
model = sm.OLS(Y6, X)
est = model.fit()
#est.summary()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.024620389354870986

In [41]:
coef_7 = coef(Y7)
# Question 3
model = sm.OLS(Y4, X)
est = model.fit()
#est.summary()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.01373379293728616

In [42]:
coef_8 = coef(Y8)
# Question 3
model = sm.OLS(Y4, X)
est = model.fit()
#est.summary()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.01373379293728616

In [43]:
coef_9 = coef(Y9)
# Question 3
model = sm.OLS(Y4, X)
est = model.fit()
#est.summary()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.01373379293728616

In [44]:
#Append coefficients and show estimates. 
coef_df = pd.concat([coef_1, coef_2, coef_3, coef_4, coef_5, coef_6, coef_7, coef_8, coef_9])
coef_9

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,-0.0157,0.017,-0.940,0.347,-0.049,0.017
Form,-0.0150,0.014,-1.058,0.290,-0.043,0.013
Both,-0.0518,0.016,-3.184,0.001,-0.084,-0.020


## 5. Discrimination

In [45]:
Y1 = df['d_discrimination']


In [58]:
any_treatment_grp[['d_discrimination']].mean()


,d_discrimination
any_treatment,
0.0,0.203158
1.0,0.240733


In [46]:
coef(Y1)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0677,0.021,3.170,0.002,0.026,0.110
Form,0.0231,0.018,1.278,0.201,-0.012,0.059
Both,0.0170,0.021,0.819,0.413,-0.024,0.058


In [47]:
model = sm.OLS(Y1, X)
est = model.fit()
#est.summary()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.06519767925707742

In [48]:
df['school']=0
df.loc[df['primary_school'] == 1, 'school'] = 1 
df.loc[df['high_school'] == 1, 'school'] = 1 

In [49]:
#School 
df1=df.loc[df['school'] == 1]
df1.groupby(['any_treatment'])['d_discrimination'].mean()

any_treatment
0.0    0.143791
1.0    0.163816
Name: d_discrimination, dtype: float64

In [50]:
#Technical education 
df2=df.loc[df['technical_educ'] == 1]
df2.groupby(['any_treatment'])['d_discrimination'].mean()


any_treatment
0.0    0.177305
1.0    0.243867
Name: d_discrimination, dtype: float64

In [51]:
#Undergraduate
df3=df.loc[df['undergraduate'] == 1]
df3.groupby(['any_treatment'])['d_discrimination'].mean()

any_treatment
0.0    0.320611
1.0    0.336158
Name: d_discrimination, dtype: float64

In [52]:
#Postgraduate
df4=df.loc[df['postgraduate'] == 1]
df4.groupby(['any_treatment'])['d_discrimination'].mean()

any_treatment
0.0    0.448718
1.0    0.502646
Name: d_discrimination, dtype: float64

In [37]:
def model(data):
    Y = data['d_discrimination']
    D1 = data[['any_video']]
    D2 = data[['any_form']]
    D3 = data[['any_video_form']]
    D = D1.join(D2)
    D = D.join(D3)
    C = data[['age', 'female', 'bog_dept']]
    X = D.join(C)
    X = sm.add_constant(X)
    reg = sm.OLS(Y, X)
    est = reg.fit()  
    return est.summary()

In [54]:
model(df1)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       d_discrimination   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     2.783
Date:                Fri, 10 Jun 2022   Prob (F-statistic):             0.0108
Time:                        12:14:06   Log-Likelihood:                -635.79
No. Observations:                1570   AIC:                             1286.
Df Residuals:                    1563   BIC:                             1323.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.1358      0.034      3.936      0.000       0.068       0.203
any_video          0.0907      0.028      3.259      0.001       0.036       0.145
any_form          -0.0024      0.023     -0.104      0.917      -0.048       0.043
any_video_form    -0.0013      0.027     -0.048      0.962      -0.054       0.052
age            -7.571e-05      0.001     -0.090      0.929      -0.002       0.002
female             0.0251      0.019      1.350      0.177      -0.011       0.062
bog_dept          -0.0081      0.019     -0.435      0.663      -0.045       0.028
==============================================================================
Omnibus:                      483.143   Durbin-Watson:                   1.852
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1041.625
Skew:                           1.849   Prob(JB):                    6.52e-227
Kurtosis:                       4.501   Cond. No.                         149.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [55]:
model(df2)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       d_discrimination   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     2.861
Date:                Fri, 10 Jun 2022   Prob (F-statistic):            0.00911
Time:                        12:14:17   Log-Likelihood:                -522.86
No. Observations:                 975   AIC:                             1060.
Df Residuals:                     968   BIC:                             1094.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.0569      0.051      1.113      0.266      -0.043       0.157
any_video          0.0650      0.041      1.589      0.112      -0.015       0.145
any_form           0.0664      0.034      1.978      0.048       0.001       0.132
any_video_form     0.0583      0.039      1.503      0.133      -0.018       0.134
age                0.0039      0.001      3.040      0.002       0.001       0.006
female             0.0263      0.027      0.985      0.325      -0.026       0.079
bog_dept          -0.0303      0.027     -1.138      0.255      -0.083       0.022
==============================================================================
Omnibus:                      172.011   Durbin-Watson:                   1.857
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              271.519
Skew:                           1.287   Prob(JB):                     1.10e-59
Kurtosis:                       2.760   Cond. No.                         150.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [56]:
model(df3)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       d_discrimination   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     3.159
Date:                Fri, 10 Jun 2022   Prob (F-statistic):            0.00477
Time:                        12:14:22   Log-Likelihood:                -313.51
No. Observations:                 485   AIC:                             641.0
Df Residuals:                     478   BIC:                             670.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.0801      0.082      0.975      0.330      -0.081       0.242
any_video          0.0125      0.060      0.207      0.836      -0.106       0.131
any_form          -0.0003      0.056     -0.004      0.996      -0.110       0.110
any_video_form     0.0101      0.063      0.161      0.872      -0.113       0.133
age                0.0065      0.002      3.875      0.000       0.003       0.010
female             0.0815      0.043      1.893      0.059      -0.003       0.166
bog_dept          -0.0461      0.043     -1.073      0.284      -0.130       0.038
==============================================================================
Omnibus:                     2229.338   Durbin-Watson:                   2.042
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               75.159
Skew:                           0.681   Prob(JB):                     4.78e-17
Kurtosis:                       1.635   Cond. No.                         179.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [57]:
model(df4)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       d_discrimination   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     2.914
Date:                Fri, 10 Jun 2022   Prob (F-statistic):            0.00910
Time:                        12:14:27   Log-Likelihood:                -185.01
No. Observations:                 267   AIC:                             384.0
Df Residuals:                     260   BIC:                             409.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.1150      0.119      0.968      0.334      -0.119       0.349
any_video          0.0249      0.090      0.276      0.783      -0.153       0.203
any_form           0.0324      0.078      0.417      0.677      -0.120       0.185
any_video_form    -0.0198      0.086     -0.229      0.819      -0.190       0.151
age                0.0099      0.003      3.907      0.000       0.005       0.015
female             0.0129      0.061      0.211      0.833      -0.107       0.133
bog_dept          -0.0763      0.062     -1.236      0.217      -0.198       0.045
==============================================================================
Omnibus:                     1651.833   Durbin-Watson:                   1.780
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               34.074
Skew:                           0.036   Prob(JB):                     3.99e-08
Kurtosis:                       1.251   Cond. No.                         189.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 6. Ambiguity

In [26]:
any_treatment_grp[['d_ambiguity_order_necessity', 'd_ambiguity_aversion', 'd_ambiguity_decision']].mean()


,d_ambiguity_order_necessity,d_ambiguity_aversion,d_ambiguity_decision
any_treatment,,,
0.0,0.111579,0.106316,0.081053
1.0,0.099702,0.129953,0.075841


In [28]:
#Declaring the variables of the model
D1 = df[['any_video']]
D2 = df[['any_form']]
D3 = df[['any_video_form']]
D = D1.join(D2)
D = D.join(D3)
X = D.join(C)
X = sm.add_constant(X)

In [29]:
coef(Y1)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0691,0.022,3.118,0.002,0.026,0.113
Form,-0.0275,0.019,-1.470,0.142,-0.064,0.009
Both,0.0132,0.022,0.614,0.539,-0.029,0.056


In [31]:
model = sm.OLS(Y1, X)
est = model.fit()
#est.summary()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.09168177232876362

In [32]:
coef(Y2) 

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0397,0.020,1.982,0.048,0.000,0.079
Form,0.0020,0.017,0.115,0.908,-0.031,0.035
Both,0.0364,0.019,1.868,0.062,-0.002,0.075


In [35]:
model = sm.OLS(Y2, X)
est = model.fit()
#est.summary()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.013860754118361851

In [34]:
coef(Y3)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,0.0772,0.023,3.349,0.001,0.032,0.122
Form,0.0013,0.020,0.068,0.945,-0.037,0.040
Both,0.0615,0.022,2.743,0.006,0.018,0.105


In [36]:
model = sm.OLS(Y3, X)
est = model.fit()
#est.summary()
r2 = est.rsquared
obs = len(est.fittedvalues)  
r2

0.12324740426088221

## 7. Dehumanization

In [39]:
Y1 = df['d_negative_emotions_left']
Y2 = df['d_negative_emotions_center']
Y3 = df['d_negative_emotions_right']

In [40]:
any_treatment_grp[['d_negative_emotions_left', 'd_negative_emotions_center', 'd_negative_emotions_right']].mean()

,d_negative_emotions_left,d_negative_emotions_center,d_negative_emotions_right
any_treatment,,,
0.0,0.660000,0.676842,0.696842
1.0,0.632297,0.670217,0.668939


In [42]:
coef(Y1)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,-0.0535,0.025,-2.140,0.032,-0.102,-0.004
Form,0.0032,0.021,0.153,0.879,-0.038,0.045
Both,-0.0542,0.024,-2.232,0.026,-0.102,-0.007


In [43]:
model = sm.OLS(Y1, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)
r2

0.016729395819085724

In [44]:
coef(Y2)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,-0.0145,0.024,-0.599,0.549,-0.062,0.033
Form,0.0068,0.020,0.331,0.741,-0.033,0.047
Both,0.0008,0.024,0.032,0.974,-0.045,0.047


In [45]:
model = sm.OLS(Y2, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)
r2

0.03342414980317554

In [46]:
coef(Y3)

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
Video,-0.0432,0.024,-1.781,0.075,-0.091,0.004
Form,-0.0063,0.021,-0.305,0.760,-0.046,0.034
Both,-0.0287,0.024,-1.216,0.224,-0.075,0.018


In [47]:
model = sm.OLS(Y3, X)
est = model.fit()
r2 = est.rsquared
obs = len(est.fittedvalues)
r2

0.024386382445118593